<a href="https://colab.research.google.com/github/renzungo/Clarin_Covers_Sent_Analysis/blob/sentiment/06_aggregate_and_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive; drive.mount('/content/drive', force_remount=True)
!pip -q install pandas numpy

import os, json, pandas as pd, numpy as np

WORK_DIR = "/content/drive/MyDrive/Data Justicialista/Clarin Cover Sentiment Analysis/odc_pipeline_work"
OUT_DIR  = "/content/drive/MyDrive/Data Justicialista/Clarin Cover Sentiment Analysis/odc_analytics_out"
os.makedirs(OUT_DIR, exist_ok=True)

def exists(p):
    return os.path.exists(p) and os.path.getsize(p) > 0

# ---- Base texts (needed for the file/date backbone)
base_path_parquet = os.path.join(WORK_DIR, "base.parquet")
if not exists(base_path_parquet):
    raise RuntimeError("base.parquet not found. Run notebook 01 first.")
df_base = pd.read_parquet(base_path_parquet)[["file","date"]]

# ---- Topics (parquet or csv rows)
topics_parquet = os.path.join(WORK_DIR, "topics.parquet")
topics_rows_csv = os.path.join(WORK_DIR, "topics_rows.csv")  # optional streaming format if you switch 02 to rows

if exists(topics_parquet):
    df_topics = pd.read_parquet(topics_parquet)[["file","date","topics_json","top_topic","top_topic_score"]]
elif exists(topics_rows_csv):
    df_topics = pd.read_csv(topics_rows_csv)[["file","date","topics_json","top_topic","top_topic_score"]]
else:
    df_topics = pd.DataFrame(columns=["file","date","topics_json","top_topic","top_topic_score"])

# ---- Sentiment overall / eco-gov (parquet or csv rows)
sent_parquet = os.path.join(WORK_DIR, "sentiment.parquet")
sent_rows_csv = os.path.join(WORK_DIR, "sentiment_rows.csv")  # optional streaming format if you switch 03 to rows

if exists(sent_parquet):
    df_sent = pd.read_parquet(sent_parquet)[["file","date","overall_sentiment","overall_score","eco_share","eco_sentiment","eco_sent_score","gov_share","gov_sentiment","gov_sent_score"]]
elif exists(sent_rows_csv):
    df_sent = pd.read_csv(sent_rows_csv)[["file","date","overall_sentiment","overall_score","eco_share","eco_sentiment","eco_sent_score","gov_share","gov_sentiment","gov_sent_score"]]
else:
    df_sent = pd.DataFrame(columns=["file","date","overall_sentiment","overall_score","eco_share","eco_sentiment","eco_sent_score","gov_share","gov_sentiment","gov_sent_score"])

# ---- NER counts (from step 04)
ner_counts_parquet = os.path.join(WORK_DIR, "ner_counts.parquet")
if exists(ner_counts_parquet):
    df_counts = pd.read_parquet(ner_counts_parquet)[["file","date","people_json","orgs_json","places_json"]]
else:
    df_counts = pd.DataFrame(columns=["file","date","people_json","orgs_json","places_json"])

# ---- Lexicon top words (from 01)
lex_parquet = os.path.join(WORK_DIR, "lexicon.parquet")
if exists(lex_parquet):
    df_lex = pd.read_parquet(lex_parquet)[["file","date","top_pos_words_json","top_neg_words_json"]]
else:
    df_lex = pd.DataFrame(columns=["file","date","top_pos_words_json","top_neg_words_json"])

# ---- Entities (growing CSV from streaming 05)
entities_csv = os.path.join(OUT_DIR, "entities_sentiment.csv")
if exists(entities_csv):
    df_entities = pd.read_csv(entities_csv)
else:
    df_entities = pd.DataFrame(columns=["file","date","entity_norm","entity_type","mentions","avg_score","pos_share","neu_share","neg_share"])

# ---- N-grams
ngrams_csv = os.path.join(OUT_DIR, "ngrams.csv")
if exists(ngrams_csv):
    df_ngrams = pd.read_csv(ngrams_csv)
else:
    df_ngrams = pd.DataFrame(columns=["file","date","ngram","n","count"])

# ---- Build master (left-join whatever we have)
df_master = (df_base
             .merge(df_sent,   on=["file","date"], how="left")
             .merge(df_topics, on=["file","date"], how="left")
             .merge(df_counts, on=["file","date"], how="left")
             .merge(df_lex,    on=["file","date"], how="left"))

# Ensure columns exist
for col in ["topics_json","people_json","orgs_json","places_json","top_pos_words_json","top_neg_words_json",
            "overall_sentiment","overall_score","eco_share","eco_sentiment","eco_sent_score","gov_share","gov_sentiment","gov_sent_score"]:
    if col not in df_master.columns: df_master[col] = np.nan

master_csv = os.path.join(OUT_DIR, "covers_master.csv")
df_master.to_csv(master_csv, index=False, encoding="utf-8")
print("Wrote:", master_csv)

# ---- Timeseries: sentiment by date (based on whatever has overall_sentiment)
def mean_share(series, target):
    if series.empty: return np.nan
    vals = [1 if x==target else 0 for x in series]
    return np.mean(vals) if len(vals) else np.nan

ts_sent = (df_master.dropna(subset=["date"])
           .groupby("date", as_index=False)
           .agg(overall_pos=("overall_sentiment", lambda s: mean_share(s, "POS")),
                overall_neu=("overall_sentiment", lambda s: mean_share(s, "NEU")),
                overall_neg=("overall_sentiment", lambda s: mean_share(s, "NEG")),
                eco_share=("eco_share","mean"),
                gov_share=("gov_share","mean")))

ts_sent_csv = os.path.join(OUT_DIR, "timeseries_sentiment.csv")
ts_sent.to_csv(ts_sent_csv, index=False, encoding="utf-8")

# ---- Topic timeseries (if any topics exist yet)
def explode_topics(df):
    rows = []
    for r in df.itertuples(index=False):
        try:
            topics = json.loads(r.topics_json) if pd.notna(r.topics_json) else {}
        except Exception:
            topics = {}
        for k,v in topics.items():
            rows.append({"date": r.date, "file": r.file, "topic": k, "score": v})
    return pd.DataFrame(rows)

df_topics_ex = explode_topics(df_master)
if df_topics_ex.empty:
    topic_ts = pd.DataFrame(columns=["date","topic","avg_topic_score","covers"])
else:
    topic_ts = (df_topics_ex.dropna(subset=["date"])
                .groupby(["date","topic"], as_index=False)
                .agg(avg_topic_score=("score","mean"),
                     covers=("file","count")))

topic_ts_csv = os.path.join(OUT_DIR, "timeseries_topics.csv")
topic_ts.to_csv(topic_ts_csv, index=False, encoding="utf-8")

# ---- People timeseries (works with partial entities)
if not df_entities.empty:
    people_ts = (df_entities[df_entities["entity_type"]=="PER"]
                 .dropna(subset=["date"])
                 .groupby(["date","entity_norm"], as_index=False)
                 .agg(mentions=("mentions","sum"),
                      pos_share=("pos_share","mean"),
                      neg_share=("neg_share","mean")))
else:
    people_ts = pd.DataFrame(columns=["date","entity_norm","mentions","pos_share","neg_share"])

people_ts_csv = os.path.join(OUT_DIR, "timeseries_people.csv")
people_ts.to_csv(people_ts_csv, index=False, encoding="utf-8")

print("Also wrote:")
print(ts_sent_csv)
print(topic_ts_csv)
print(people_ts_csv)


Mounted at /content/drive
Wrote: /content/drive/MyDrive/Data Justicialista/Clarin Cover Sentiment Analysis/odc_analytics_out/covers_master.csv
Also wrote:
/content/drive/MyDrive/Data Justicialista/Clarin Cover Sentiment Analysis/odc_analytics_out/timeseries_sentiment.csv
/content/drive/MyDrive/Data Justicialista/Clarin Cover Sentiment Analysis/odc_analytics_out/timeseries_topics.csv
/content/drive/MyDrive/Data Justicialista/Clarin Cover Sentiment Analysis/odc_analytics_out/timeseries_people.csv
